## Assignment 2

In this assignment, we are asking you to make estimates for what to expect when doing orbit integrations. This will rely on some of the math that we went over at the first meeting. You should be able to find those slides on the StreamTeam Google Drive, which is linked in the Slack "General" channel bookmarks (above all the messages).

Using some of the equations that we learned about last week, we should be able to make predictions about the orbits of individual stars. This will help us understand how streams form and also give allow us to compare our code results to expectations once we start to make streams in Gala.

I will guide you through writing the code for the first question and you should use the coding techniques (commenting, printing answers, keeping track of units, function writing (optional)) introduced there when finding the answers to questions 2 and 3.

First let's import the normal packages that we always use

In [11]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Gala (this is not strictly necessary, since this notebook doesn't use Gala, but a good habit)
import gala
import gala.dynamics as gd
import gala.potential as gp

#### Quick Background

As we have seen, streams are made up of stars that were once either dwarf galaxies or globular clusters. We can refer to either one of these as "satellites" that orbit the Milky Way. On those orbits, satellite stars are *tidally stripped*, forming streams. The stars are stripped when the "pull" from the host galaxy is stronger than the gravitational force within the satellite itself. When that happens to a star, it will separate from its satellite and become part of a stream of stars undergoing the same fate. THe overcoming of the gravitational force of the satellite on a given star will happen at a certain radius from the center of the satellite, called the *tidal radius*. The *tidal radius* at which stars are lost from the satellite is given by the formula $$r_{tide} \sim (\frac{m_{sat}}{M_{gal}})^{1/3} R$$

#### Question 1

Calculate $r_{tide}$ for two satellites, one with mass $m_{sat} = 10^5 M_{\odot}$ and one with mass $m_{sat} = 10^9 M_{\odot}$ orbiting a galaxy of mass $M_{gal} = 5 \times 10^{11} M_{\odot}$ at $R_0 = 30$ kpc

In [1]:
# write code below to calculate r_tide. 
#  Call r_tide for the smaller satellite r_tide1 and r_tide for the larger one r_tide2
r_tide1 = ((10**5 / 5e11)**(1/3)) * 30
r_tide2 = ((10**9 / 5e11)**(1/3)) * 30

# Quick check: What units is your answer in? 
    #ans: kpc, because R is in kpc and the m/M term is unitless
#  You didn't need to code units into your code specifically, 
#   but you should always make sure you know what units you are working with. 
#  If it isn't already, make sure that r_tide1 and r_tide2 are in kpc, 
#   so that the two lines of output at the end of this cell are correct

# End your code with these two lines to output your results:
print('Smaller satellite r_tide = {} kpc'.format(r_tide1))
print('Larger satellite r_tide = {} kpc'.format(r_tide2))

Smaller satellite r_tide = 0.175441064292772 kpc
Larger satellite r_tide = 3.7797631496846193 kpc


Now that you've written your code let's do a guided practice with function writing in python if you aren't familiar with it already. Try to write a function that takes in as variables the satelite mass, galaxy mass, and radius from galaxy center, and outputs the tidal radius.

In [3]:
# Here is the basic structure of your code to write functions:

# Make sure to change the names of the functions and parameters to things 
#  that make sense for you and anyone reading your code
    
def tidal_radius(m_sat, M_gal, R): 
    # use the code you've written above to go from your inputed parameters to the output
    r_tide = ((m_sat / M_gal)**(1/3)) * R
    return r_tide # this outputs you the tidal radius for your inputed parameters

As a test, you should now be able to run the following cell, after changing the name of your function

In [4]:
r_tide1 = tidal_radius(10**5, 5*10**11, 30)
r_tide2 = tidal_radius(10**9, 5*10**11, 30)

print('Smaller satellite r_tide = {} kpc'.format(r_tide1))
print('Larger satellite r_tide = {} kpc'.format(r_tide2))

# this should output exactly the same thing as your initial code 
#  but it has the advantage that you can now easily try out other values for m_sat, m_gal, and R

Smaller satellite r_tide = 0.175441064292772 kpc
Larger satellite r_tide = 3.7797631496846193 kpc


Pick one of the two satellites and use the function you just wrote to answer: What happens when to $r_{tide}$ when you decrease the radius of the satellite to 5 kpc? What if you increase it to 100 kpc?

In [5]:
# Write your code below using the function you defined earlier

r_tide_inner = tidal_radius(10**5, 5e11, 5)
r_tide_outer = tidal_radius(10**5, 5e11, 100)

print('Inner satellite r_tide = {} kpc'.format(r_tide_inner))
print('Outer satellite r_tide = {} kpc'.format(r_tide_outer))

Inner satellite r_tide = 0.029240177382128668 kpc
Outer satellite r_tide = 0.5848035476425734 kpc


#### Question 2

From the previous assignment, you should have some functions written for the Hernquist potential using the Keplerian potential used in "Leapfrog_tutorial" as a guide. Apply those functions for the potential, acceleration, and circular velocity to complete this question. 

For each satellite from the previous question, what is the difference in the orbital period $T$ for stars that end up in the leading (at $R = R_0 + r_{tide}$) and trailing (at $R = R_0 - r_{tide}$) streams? Assume that the satellites are on a circular orbit, meaning you can use one of the formulas for $T$ in the slides (or at the end of the "Leapfrog_tutorial" notebook).

What is the difference in energy for those two galaxies? Use the formula for total energy $$E = \frac{1}{2} v_{circ}^2 + \Phi(R \pm r_{tide})$$ where $\Phi$ represents the Hernquist potential.

In [6]:
# Your code below...

#functions -- all return SI units (sorry), assuming solar mass & kpc inputs
G = 6.6743e-11
def hernquist_potential(M, r, c):
    r_to_m = r * 3.086e19 #convert kpc input to meters
    M_to_kg = M * 1.989e30 #convert solar mass input to kilograms
    return (-G*M_to_kg)/(r_to_m + c) 

def hernquist_acceleration(M, r, c):
    r_to_m = r * 3.086e19 #convert kpc input to meters
    M_to_kg = M * 1.989e30 #convert solar mass input to kilograms
    return (-G * M_to_kg) / ((r_to_m + c)**2)

def circular_velocity(M, r, c):
    M_to_kg = M * 1.989e30
    r_to_m = r * 3.086e10
    return np.sqrt((G*M_to_kg*r_to_m) / (r_to_m + c)**2)

def orbital_period(M, r, c):
    v_circ = circular_velocity(M, r, c)
    r_to_m = r * 3.086e10
    return (2*np.pi*r_to_m) / v_circ 


In [15]:
#smaller sattelite
r_tide1 = tidal_radius(10**5, 5e11, 30) #kpc
r_tide1_leading = tidal_radius(10**5, 5e11, 30 + r_tide1) #kpc
r_tide1_trailing = tidal_radius(10**5, 5e11, 30 - r_tide1) #kpc

#larger sattelite
r_tide2 = tidal_radius(10**9, 5e11, 30) #kpc
r_tide2_leading = tidal_radius(10**9, 5e11, 30 + r_tide1) #kpc
r_tide2_trailing = tidal_radius(10**9, 5e11, 30 - r_tide1) #kpc

#check
print('Smaller sattelite leading radius: {}'.format(r_tide1_leading) + ' kpc')
print('Smaller sattelite trailing radius: {}'.format(r_tide1_trailing) + ' kpc')
print('Larger sattelite leading radius: {}'.format(r_tide2_leading) + ' kpc')
print('Larger sattelite trailing radius: {}'.format(r_tide2_trailing) + ' kpc')

Smaller sattelite leading radius: 0.17646704986077805 kpc
Smaller sattelite trailing radius: 0.17441507872476597 kpc
Larger sattelite leading radius: 3.801867338676462 kpc
Larger sattelite trailing radius: 3.757658960692777 kpc


In [30]:
#smaller sattelite periods
period1_leading = orbital_period(10**5, r_tide1_leading, 1)
period1_trailing = orbital_period(10**5, r_tide1_trailing, 1)
diff_period1 = period1_leading - period1_trailing

#larger sattelite periods
period2_leading = orbital_period(10**9, r_tide2_leading, 1)
period2_trailing = orbital_period(10**9, r_tide2_trailing, 1)
diff_period2 = period2_leading - period2_trailing

delta = diff_period1 - diff_period2

#results
print('Leading-trailing period difference for smaller sattelite: {} s'.format(diff_period1))
print('Leading-trailing period difference for larger sattelite: {} s'.format(diff_period2))
print('Difference between smaller and larger sattelite gaps: {} s'.format(delta))

Leading-trailing period difference for smaller sattelite: 12.05260744892098 s
Leading-trailing period difference for larger sattelite: 12.05260744821328 s
Difference between smaller and larger sattelite gaps: 7.077005648170598e-10 s


In [22]:
#energies
R_to_m = 30 * 3.086e10

#smaller
v_circ1 = circular_velocity(10**5, 30, 1)
hernquist_pot1 = hernquist_potential(10**5, 30, 1)
r_tide1_m = r_tide1 * 3.086e10

energy1_leading = (0.5 * (v_circ1**2)) + (hernquist_pot1 + (R_to_m + r_tide1_m))
energy1_trailing = (0.5 * (v_circ1**2)) + (hernquist_pot1 + (R_to_m - r_tide1_m))

diff_energy1 = energy1_leading - energy1_trailing

#larger
v_circ2 = circular_velocity(10**9, 30, 1)
hernquist_pot2 = hernquist_potential(10**9, 30, 1)
r_tide2_m = r_tide2 * 3.086e10

energy2_leading = (0.5 * (v_circ2**2)) + (hernquist_pot2 + (R_to_m + r_tide2_m))
energy2_trailing = (0.5 * (v_circ2**2)) + (hernquist_pot2 + (R_to_m - r_tide2_m))

diff_energy2 = energy2_leading - energy2_trailing

#results
print('Leading-trailing energy difference for smaller sattelite: {} kJ'.format(diff_energy1 / 1000))
print('Leading-trailing energy difference for larger sattelite: {} kJ'.format(diff_energy2 / 1000))

Leading-trailing energy difference for smaller sattelite: 10828222.48815039 kJ
Leading-trailing energy difference for larger sattelite: 233286981.6 kJ


#### Question 3

Roughly how long (in angle around the galaxy) do you think your streams should be after 1 orbit? After 10 orbits?

Hint: You'll need to find the angular velocity difference of stars in the leading and trailing streams from your answer to **Question 2**.

In [23]:
# Your code below...

#delta v_ang = angle / time ==> angle = v_ang * time

#N = number of orbits
def gal_angle(delta_v_ang, delta_period, N):
    return delta_v_ang * (delta_period * N) #returns in radians

In [34]:
#smaller
v_ang_leading1 = 2*np.pi / period1_leading #angular velocity, units rad/s
v_ang_trailing1 = 2*np.pi / period1_trailing

delta_v_ang1 = abs(v_ang_leading1 - v_ang_trailing1) #angular velocity difference

#larger
v_ang_leading2 = 2*np.pi / period2_leading
v_ang_trailing2 = 2*np.pi / period2_trailing

delta_v_ang2 = abs(v_ang_leading2 - v_ang_trailing2)

#calculations
angle1_one = gal_angle(delta_v_ang1, diff_period1, 1)
angle1_ten = gal_angle(delta_v_ang1, diff_period1, 10)

angle2_one = gal_angle(delta_v_ang2, diff_period2, 1)
angle2_ten = gal_angle(delta_v_ang2, diff_period2, 10)

#print results 
print('Smaller sattelite angular size, one orbit: {0} rads, ten orbits: {1} rads'.format(angle1_one, angle1_ten))
print('Larger sattelite angular size, one orbit: {0} rads, ten orbits: {1} rads'.format(angle2_one, angle2_ten))


Smaller sattelite angular size, one orbit: 0.001934030935274034 rads, ten orbits: 0.01934030935274034 rads
Larger sattelite angular size, one orbit: 0.0019340309357282065 rads, ten orbits: 0.019340309357282065 rads


In [36]:
#in degrees
angle1_one_deg = angle1_one * (180 / np.pi)
angle1_ten_deg = angle1_ten * (180 / np.pi)

angle2_one_deg = angle2_one * (180 / np.pi)
angle2_ten_deg = angle2_ten * (180 / np.pi)

print('Smaller sattelite, one orbit: {0} deg, ten orbits: {1} deg'.format(angle1_one_deg, angle1_ten_deg))
print('Smaller sattelite, one orbit: {0} deg, ten orbits: {1} deg'.format(angle2_one_deg, angle2_ten_deg))


Smaller sattelite, one orbit: 0.11081181003894144 deg, ten orbits: 1.1081181003894145 deg
Smaller sattelite, one orbit: 0.11081181006496361 deg, ten orbits: 1.1081181006496361 deg
